# CVRP

In [6]:
import pandas as pd
import folium
import requests
import time
from folium.features import CustomIcon

In [7]:
time_sheet = pd.read_excel('Routing problem - Raw Data - V2 .xlsx', index_col='Code',sheet_name='Time')

In [8]:
time_matrix = [time_sheet.iloc[i].values.tolist() for i in range(43)]

In [15]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['time_matrix'] = time_matrix
    data['demands'] = [0, 1, 1 ,0] + [1] * 40
    data['vehicle_capacities'] = [5, 5] + [4] * 15
    data['num_vehicles'] = 17
    data['starts'] =[1, 2] + [3] * 15
    data['ends'] = [0 for _ in range(17)]
    return data

def print_solution(data, manager, routing, solution):
     """Prints solution on console."""
     print(f'Objective: {solution.ObjectiveValue()}')
     total_distance = 0
     total_load = 0
     for vehicle_id in range(data['num_vehicles']):
         index = routing.Start(vehicle_id)
         plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
         route_distance = 0
         route_load = 0
         while not routing.IsEnd(index):
             node_index = manager.IndexToNode(index)
             route_load += data['demands'][node_index]
             plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
             previous_index = index
             index = solution.Value(routing.NextVar(index))
             route_distance += routing.GetArcCostForVehicle(
                 previous_index, index, vehicle_id)
         plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
         route_load)
         plan_output += 'Time of the route: {} minutes'.format(route_distance)
         plan_output += 'Load of the route: {}\n '.format(route_load)
         print(plan_output)
         total_distance += route_distance
         total_load += route_load
     print('Total time of all routes: {} minutes'.format(total_distance))
     print('Total load of all routes: {}'.format(total_load))
     print('-'* 20)


def main():
     """Solve the CVRP problem."""
     # Instantiate the data problem.
     data = create_data_model()
     # Create the routing index manager.
     manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']),
                                             data['num_vehicles'], data['starts'],
                                             data['ends'])
                                             # Create Routing Model.
     routing = pywrapcp.RoutingModel(manager)
     # Create and register a transit callback.
     def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]

     transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
     routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


     # Add Capacity constraint.
     def demand_callback(from_index):
         """Returns the demand of the node."""
         # Convert from routing variable Index to demands NodeIndex.
         from_node = manager.IndexToNode(from_index)
         return data['demands'][from_node]
         
     demand_callback_index = routing.RegisterUnaryTransitCallback(
     demand_callback)
     routing.AddDimensionWithVehicleCapacity(
         demand_callback_index,
         0, # null capacity slack
         data['vehicle_capacities'], # vehicle maximum capacities
         True, # start cumul to zero
         'Capacity')

     # Setting first solution heuristic.
     search_parameters = pywrapcp.DefaultRoutingSearchParameters()
     search_parameters.first_solution_strategy = (
     routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
     search_parameters.local_search_metaheuristic = (
     routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
     search_parameters.time_limit.FromSeconds(1)
     # Solve the problem.
     solution = routing.SolveWithParameters(search_parameters)
     # Print solution on console.
     if solution:
         print_solution(data, manager, routing, solution)

if __name__ == '__main__':
    main()

Objective: 785
Route for vehicle 0:
 1 Load(1) ->  12 Load(2) ->  10 Load(3) ->  35 Load(4) ->  9 Load(5) ->  0 Load(5)
Time of the route: 85 minutesLoad of the route: 5
 
Route for vehicle 1:
 2 Load(1) ->  31 Load(2) ->  40 Load(3) ->  5 Load(4) ->  16 Load(5) ->  0 Load(5)
Time of the route: 88 minutesLoad of the route: 5
 
Route for vehicle 2:
 3 Load(0) ->  26 Load(1) ->  38 Load(2) ->  15 Load(3) ->  33 Load(4) ->  0 Load(4)
Time of the route: 99 minutesLoad of the route: 4
 
Route for vehicle 3:
 3 Load(0) ->  27 Load(1) ->  32 Load(2) ->  6 Load(3) ->  11 Load(4) ->  0 Load(4)
Time of the route: 117 minutesLoad of the route: 4
 
Route for vehicle 4:
 3 Load(0) ->  22 Load(1) ->  13 Load(2) ->  19 Load(3) ->  17 Load(4) ->  0 Load(4)
Time of the route: 101 minutesLoad of the route: 4
 
Route for vehicle 5:
 3 Load(0) ->  8 Load(1) ->  14 Load(2) ->  7 Load(3) ->  4 Load(4) ->  0 Load(4)
Time of the route: 107 minutesLoad of the route: 4
 
Route for vehicle 6:
 3 Load(0) ->  0 Lo

In [16]:
df = pd.read_excel('Routing problem - Raw Data - V2 .xlsx')

In [17]:
dataframe = {'Latitude':df['Latitude'].tolist(),
            'Longitude':df['Longitude'].tolist(),
            'Gender':df['Gender'].tolist(),
            'Name':df['Family'].tolist()}
dataframe = pd.DataFrame(dataframe)

In [18]:
cordinations_list = [[1, 12, 10, 35, 9, 0],
                    [2, 4, 7, 14, 8, 0],
                    [3, 26, 38, 15, 33, 0],
                    [3, 27, 32, 11, 6, 0],
                    [3, 16, 5, 40, 31, 0],
                    [3, 22, 13, 19, 17, 0],
                    [3, 37, 24, 30, 25, 0],
                    [3, 36, 41, 34, 39, 0],
                    [3, 18, 28, 21, 42, 0],
                    [3, 23, 29, 20, 0]]

In [19]:
cordinations = [dataframe[['Latitude', 'Longitude']].iloc[[1, 12, 10, 35, 9, 0]].values.tolist(),
               dataframe[['Latitude', 'Longitude']].iloc[[2, 4, 7, 14, 8, 0]].values.tolist(),
               dataframe[['Latitude', 'Longitude']].iloc[[3, 26, 38, 15, 33, 0]].values.tolist(),
               dataframe[['Latitude', 'Longitude']].iloc[[3, 27, 32, 11, 6, 0]].values.tolist(),
               dataframe[['Latitude', 'Longitude']].iloc[[3, 16, 5, 40, 31, 0]].values.tolist(),
               dataframe[['Latitude', 'Longitude']].iloc[[3, 22, 13, 19, 17, 0]].values.tolist(),
               dataframe[['Latitude', 'Longitude']].iloc[[3, 37, 24, 30, 25, 0]].values.tolist(),
               dataframe[['Latitude', 'Longitude']].iloc[[3, 36, 41, 34, 39, 0]].values.tolist(),
               dataframe[['Latitude', 'Longitude']].iloc[[3, 18, 28, 21, 42, 0]].values.tolist(),
               dataframe[['Latitude', 'Longitude']].iloc[[3, 23, 29, 20, 0]].values.tolist(),]

In [21]:
for index, points in enumerate(cordinations):
    
    dataframe = {'Latitude':df['Latitude'].iloc[cordinations_list[index]].tolist(),
            'Longitude':df['Longitude'].iloc[cordinations_list[index]].tolist(),
            'Gender':df['Gender'].iloc[cordinations_list[index]].tolist(),
            'Name':df['Family'].iloc[cordinations_list[index]].tolist()}
    dataframe = pd.DataFrame(dataframe)
    
    m = folium.Map(location=[36.060047, 50.530882], zoom_start=15)

    # Add markers for each location with custom icons
    for index_dataframe, row in dataframe.iterrows():
        if index_dataframe == (dataframe.shape[0] - 1):
            folium.Marker(
                location=[row['Latitude'], row['Longitude']],
                popup=row['Name'],
                icon=folium.Icon(color="green", icon='home', prefix='fa')
        ).add_to(m)

        elif index_dataframe == 0:
            folium.Marker(
                location=[row['Latitude'], row['Longitude']],
                popup=row['Name'],
                icon=folium.Icon(color="red", icon='car', prefix='fa')
        ).add_to(m)
            
        elif row['Gender'] == 'M':
            folium.Marker(
                location=[row['Latitude'], row['Longitude']],
                popup=row['Name'],
                icon=folium.Icon(color="blue", icon='person', prefix='fa', iconColor='black'),   
            ).add_to(m)
            
        else:
            folium.Marker(
                location=[row['Latitude'], row['Longitude']],
                popup=row['Name'],
                icon=folium.Icon(color="pink", icon='person-dress', prefix='fa'),  
            ).add_to(m)
    
   # Calculate routes between points using OpenRouteService API
    for i in range(len(points) - 1):
        start_point = points[i]
        end_point = points[i+1]
        
        # Request route data from OpenRouteService
        params = {
            "api_key": "5b3ce3597851110001cf6248f668245b4f504409a13a50f75764fa80",
            "start": f"{start_point[1]},{start_point[0]}",
            "end": f"{end_point[1]},{end_point[0]}"
        }
        api_url = "https://api.openrouteservice.org/v2/directions/driving-car"
        response = requests.get(api_url, params=params)
        route_data = response.json()
        
        # Extract coordinates along the route
        route_coordinates = [
            (coord[1], coord[0]) for coord in route_data["features"][0]["geometry"]["coordinates"]
        ]
        
        # Add polyline for the route
        folium.PolyLine(locations=route_coordinates, color='blue', weight=3).add_to(m)
        time.sleep(2)
        m.save(f'./map{index}.html')

# Locations of Individuals

In [22]:
df = pd.read_excel('Routing problem - Raw Data - V2 .xlsx')

In [23]:
dataframe = {'Latitude':df['Latitude'].tolist(),
            'Longitude':df['Longitude'].tolist(),
            'Gender':df['Gender'].tolist(),
            'Name':df['Family'].tolist()}
dataframe = pd.DataFrame(dataframe)

In [40]:
import folium
import requests
import time
from folium.features import CustomIcon
# Create a map centered at a specific location

m = folium.Map(location=[36.060047, 50.530882], zoom_start=15)

# Add markers for each location with custom icons
for index, row in dataframe.iterrows():
    if index == 0:
        icon = folium.Icon(icon='home', prefix='fa', color="red")
    elif index in [1, 2, 3]:
        icon = folium.Icon(icon='car', prefix='fa', color="green")
    elif row['Gender'] == 'M':
        icon = folium.Icon(color="blue", icon='person', prefix='fa', iconColor='black')
    else:
        icon = folium.Icon(color="pink", icon='female', prefix='fa')

    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Name'],
        icon=icon
    ).add_to(m)


In [42]:
m.save('./locations of individuals.html')